In [ ]:
import pandas as pd
import torch
torch.get_default_device()

In [2]:
LANGUAGE = "zho_Hans"

In [ ]:
# Run sample_1m.py to generate the samples
samples = pd.read_csv(f"samples-{LANGUAGE}-1m-200.csv").drop(columns=["Unnamed: 0"])
samples = samples[["id", "u", "ts", "text"]]

In [4]:
samples

,id,u,ts,text
0,61c0639d8f603bc66c62a91b9c47f9e3,http://www.cuishanbj.com/46013ltxfq.html,2019-03-11T03:10:27Z,"起底无限极百年酱油世家及其27年直销生意,中国石油将新建23座储气库形成六大区域储气中心,伪..."
1,3f4e99ff8a3138b7f312f04a1881ab0d,http://www.cnelc.com/Article/1/140912/AD100200...,2015-05-09T20:38:18Z,“中国电器之都”的电器文化节\n“2014中国民营企业500强”系列榜单8月18日在北京揭晓...
2,04f09536f7fe74c3b335298d0f6ce07d,http://sbsb32.com/mpyc/index.html,2019-03-16T10:15:25Z,印度空军一架军机坠毁 3人遇难 杭州保安神似岳云鹏 自学成才练出一口菱♀♀♀♀♀♀△利英语\...
3,b3f967124b6469cc6d3c5f6a21da9d2d,http://cigem.cn/auto/db/detail.aspx?db=1006&ri...,2020-09-23T07:01:53Z,"据相关负责人介绍,与往年相比,今年发布的地图主要有以下三方面特点:\n一是地图服务国家重大战..."
4,7443a6f7fc5c26969c0af16c87597f1a,http://exirgmy.ekobca002.cn/,2017-03-23T04:33:08Z,2016马报:河南荣科欣文化传播有限公司\n樊未晨/中国青年报\n03-23\n女子携8把飞...
...,...,...,...,...
195,e640affa16bb6a53dd28474224b28f9f,http://yunzhengkeji.com/yunzhengkeji_32392_692...,2018-08-16T08:47:06Z,"2016年是十三五的开局之年,也是“电改元年”。在2016年电改快速推进,政策频出超预期,各..."
196,a2527a33565e9faf7fe766db58b97e90,http://www.bestcheaplouisvuitton.com/a/news/20...,2018-05-25T03:22:06Z,内格雷特完美腾空剪刀脚扫射 散户心存侥幸\n在三峡大坝、荆江大堤、大规模拆建水域等重点流域和...
197,4c8ff5411955d1031ee284cad4709495,http://www.hnxjjc.cn/news/113.html,2023-06-10T07:33:07Z,欢迎来到湖南湘建智科工程技术有限公司!\n喜报!热烈庆祝我司获得测绘乙级资质\n- 分类:企...
198,3ac835d8d6b773e45a3c9d1c9253792d,http://news.dongyingnews.cn/system/2014/04/19/...,2014-07-05T16:52:51Z,"“宋林跌倒”,热闹背后有问号\n据中央纪委监察部网站消息,华润集团董事长、党委书记宋林涉嫌严..."


device(type='cpu')

In [7]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_id = "TurkuNLP/multilingual-web-register-classification"

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")

def infer_text(text):
    # Tokenize text
    inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    # Apply sigmoid to the logits to get probabilities
    probabilities = torch.sigmoid(outputs.logits).squeeze()

    # Determine a threshold for predicting labels
    threshold = 0.4
    predicted_label_indices = (probabilities > threshold).nonzero(as_tuple=True)[0]

    # # Extract readable labels using id2label
    id2label = model.config.id2label
    predicted_labels = [id2label[idx.item()] for idx in predicted_label_indices]
    return predicted_labels


/Users/samikukkonen/dhh25/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
from tqdm import tqdm

labels = [infer_text(record) for record in tqdm(samples['text'])]

100%|██████████| 200/200 [01:20<00:00,  2.49it/s]


In [9]:
samples_with_labels = samples.copy()
samples_with_labels["labels"] = [", ".join(label) for label in labels]

In [ ]:
samples_with_labels.head(5)

,id,u,ts,text,labels
0,61c0639d8f603bc66c62a91b9c47f9e3,http://www.cuishanbj.com/46013ltxfq.html,2019-03-11T03:10:27Z,"起底无限极百年酱油世家及其27年直销生意,中国石油将新建23座储气库形成六大区域储气中心,伪...",MT
1,3f4e99ff8a3138b7f312f04a1881ab0d,http://www.cnelc.com/Article/1/140912/AD100200...,2015-05-09T20:38:18Z,“中国电器之都”的电器文化节\n“2014中国民营企业500强”系列榜单8月18日在北京揭晓...,"NA, ne"
2,04f09536f7fe74c3b335298d0f6ce07d,http://sbsb32.com/mpyc/index.html,2019-03-16T10:15:25Z,印度空军一架军机坠毁 3人遇难 杭州保安神似岳云鹏 自学成才练出一口菱♀♀♀♀♀♀△利英语\...,"NA, ne"
3,b3f967124b6469cc6d3c5f6a21da9d2d,http://cigem.cn/auto/db/detail.aspx?db=1006&ri...,2020-09-23T07:01:53Z,"据相关负责人介绍,与往年相比,今年发布的地图主要有以下三方面特点:\n一是地图服务国家重大战...","NA, IN, ne"
4,7443a6f7fc5c26969c0af16c87597f1a,http://exirgmy.ekobca002.cn/,2017-03-23T04:33:08Z,2016马报:河南荣科欣文化传播有限公司\n樊未晨/中国青年报\n03-23\n女子携8把飞...,MT
...,...,...,...,...,...
195,e640affa16bb6a53dd28474224b28f9f,http://yunzhengkeji.com/yunzhengkeji_32392_692...,2018-08-16T08:47:06Z,"2016年是十三五的开局之年,也是“电改元年”。在2016年电改快速推进,政策频出超预期,各...",IN
196,a2527a33565e9faf7fe766db58b97e90,http://www.bestcheaplouisvuitton.com/a/news/20...,2018-05-25T03:22:06Z,内格雷特完美腾空剪刀脚扫射 散户心存侥幸\n在三峡大坝、荆江大堤、大规模拆建水域等重点流域和...,"NA, ne"
197,4c8ff5411955d1031ee284cad4709495,http://www.hnxjjc.cn/news/113.html,2023-06-10T07:33:07Z,欢迎来到湖南湘建智科工程技术有限公司!\n喜报!热烈庆祝我司获得测绘乙级资质\n- 分类:企...,"IN, IP, ds"
198,3ac835d8d6b773e45a3c9d1c9253792d,http://news.dongyingnews.cn/system/2014/04/19/...,2014-07-05T16:52:51Z,"“宋林跌倒”,热闹背后有问号\n据中央纪委监察部网站消息,华润集团董事长、党委书记宋林涉嫌严...","NA, ne"


In [11]:
samples_with_labels.to_csv(f"samples-with-labels-{LANGUAGE}-1m-200.csv")